<a href="https://colab.research.google.com/github/i-otsuka/imersao-ia-alura-google-gemini-assistente-agentes/blob/main/Assistente_Business_Party.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura ainda não anunciou uma nova edição da Imersão IA com Google Gemini. A última edição ocorreu em fevereiro de 2024.

A melhor forma de ficar sabendo sobre futuras edições é:

*   **Seguir a Alura nas redes sociais:** Eles costumam anunciar novidades por lá (LinkedIn, Instagram, etc.).
*   **Assinar a newsletter da Alura:** Assim você recebe informações diretamente no seu e-mail.
*   **Acompanhar o fórum da Alura:** A comunidade pode compartilhar informações sobre novos cursos e imersões.
*   **Verificar a página de cursos de IA da Alura:** Fique de olho na página de cursos de Inteligência Artificial da Alura para ver se há alguma novidade ou previsão de novas turmas.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Insere a tool de busca do Google ###############################################
    config={"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para o curso já estão abertas no site da Alura. O curso é gratuito, 100% online e voltado para profissionais de todas as áreas, sem exigir conhecimento prévio. Ao final da imersão, os participantes ganham um certificado da Alura e do Google.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, alura.com.br, youtube.com



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##################################
# --- Agente 1: Brainstormer --- #
##################################
def agente_brainstormer(briefing):
    brainstormer = Agent(
        name="agente_brainstormer",
        model="gemini-2.0-flash",
        # Instruções do Agente de Brainstorming #################################################
        instruction="""
          Haja como um time de brainstorming composto por diferentes perfis (O Pensador Criativo,
          O Analista, O Executor, O Entusiasta, O Cético Construtivo, O Especialista). Você deve levantar ideias
          de ações com base no briefing fornecido. O evento pode ser desde uma festa onde cada funcionário
          pode trazer um item até uma festa épica em uma supercasa de eventos. Pode ter decorações temáticas,
          convites especiais, impressos, banners, espaços instagramáveis, materiais de divulgação etc.,
          desde que se adeque ao orçamento e requisitos fornecidos.  Você pode utilizar a ferramenta do Google (google_search)
          para buscar informações sobre o tema pesquisar ideias, locais, fornecedores, brindes, materiais etc.
          Ao final elenque as 5 melhores e mais criativas ideias de festa que podem servir ao tema proposto no briefing. Inclua o
          slogan, as possíveis ações e atrações, ideias de materiais, brindes, peças de comunicação, decorações, alimentação etc.).
        """,
      description="Agente de brainstorming que gera ideias iniciais de eventos.",
      tools=[google_search]
    )
    entrada_do_agente_brainstorming = f"Briefing:{briefing}"
    lista_de_ideias = call_agent(brainstormer, entrada_do_agente_brainstorming)
    return lista_de_ideias

In [ ]:
###########################################
# --- Agente 2: Planejador de eventos --- #
###########################################
def agente_planejador(briefing, lista_de_ideias, data_de_hoje):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Você é um especialista em eventos dentro do time de comunicação interna e endomarketing da PaperCare.
          Seu objetivo é analisar as ideias iniciais levantadas na etapa de brainstorming e organizar o evento: criar a programação,
          levantar as peças de comunicação necessárias, levantar custos e elaborar o cronograma de execução (compra de itens,
          produção das peças de comunicação, minuto a minuto do evento etc.). Você pode mesclar itens de duas ou mais ideias no
          planejamento do evento. Lembre-se de considerar a data de hoje e a data do evento no planejamento. Utilize a pesquisa do
          Google (google_search) para buscar informações complementares para refinar o planejamento. Você pode contar com os canais
          de comunicação da empresa (e-mail, murais de avisos, fundo de telas, intranet e TVs) e com o time de comunicação que conta
          com pessoas que podem apoiar com desenvolvimento das artes. Ao final, gere uma apresentação com a proposta do evento. Essa apresentação
          deverá conter um resumo sobre o evento, trazer o tema e slogan, programação do evento, plano de comunicação, plano de execução
          e custos(importante especificar os nomes dos itens, quantidades, valores unitários e valores totais). Essa apresentação servirá para aprovação do evento pela gestão da empresa e será utilizada como base para criação dos
          textos e peças de comunicação pelo time de comunicação.
        """,
        description="Agente que planeja os eventos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Briefing:{briefing}\nLista de Ideias: {lista_de_ideias}\nData de início do Planejamento:{data_de_hoje}"
    # Executa o agente
    plano_do_evento = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_evento

In [ ]:
#################################
# --- Agente 3: Comunicação --- #
#################################
def agente_comunicador(briefing, plano_do_evento):
    comunicador = Agent(
        name="agente_comunicador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um especialista de comunicação da PaperCare. Sua função é elaborar os textos
            para as peças de comunicação propostas no plano do evento. As comunicações devem ser integradas
            entre si e com o core da empresa. Os textos devem levar em consideração as especificações
            fornecidas pelo briefing e plano no evento.
            Os textos devem ser de leitura rápida, engajadores, informativo, com linguagem simples e incluir 3 a 5 hashtags no final.
            Todos os materiais, preferencialmente, devem ter o mesmo título e slogan para reforçar o propósito do evento e comunicação.
            """,
        description="Agente criador dos textos das comunicações do evento."
    )
    entrada_do_agente_comunicador = f"Briefing:{briefing}\nPlano do Evento: {plano_do_evento}"
    # Executa o agente
    draft_comunicacao = call_agent(comunicador, entrada_do_agente_comunicador)
    return draft_comunicacao

In [ ]:
#######################################
# --- Agente 4: Revisor do Evento --- #
#######################################
def agente_revisor(briefing, plano_do_evento,draft_comunicacao):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é o gestor e especialista em comunicação, endomarketing e experiencia do colaborador
            da PaperCare. Sua função é revisar os drafts das comunicações e verificar se as comunicações
            estão alinhadas com o Briefing e Plano do Evento. Caso haja melhorias, sugira alterações.
            Caso esteja tudo certo, sinalize apenas com “Aprovado! Podem seguir.”
            """,
        description="Agente revisor da comunicação do evento."
    )
    entrada_do_agente_revisor = f"Briefing:{briefing}\nPlano do Evento: {plano_do_evento}\nDraft da Comunicação: {draft_comunicacao}"
    # Executa o agente
    comunicacao_revisada = call_agent(revisor, entrada_do_agente_revisor)
    return comunicacao_revisada

In [ ]:
#######################################
# --- Formulário de Briefing --- #
#######################################

def coletar_briefing():
    #Coleta informações do briefing do evento corporativo através de inputs validados.

    briefing = {}

    # Tipo de evento
    while True:
        tipo_evento = input("Qual o tipo de evento (Festa de final de ano, convenção, aniversário da empresa, campanha etc.)? ").strip()
        if tipo_evento:
            briefing['tipo_evento'] = tipo_evento
            break
        else:
            print("Por favor, informe o tipo de evento.")

    # Tema
    briefing['tema'] = input("Qual é o tema? ").strip()

    # Data/período do evento
    while True:
        data_periodo = input("Data/período do evento? ").strip()
        if data_periodo:
            briefing['data_periodo'] = data_periodo
            break
        else:
            print("Por favor, informe a data ou período do evento.")

    # Horário
    while True:
        horario = input("Horário? ").strip()
        if horario:
            briefing['horario'] = horario
            break
        else:
            print("Por favor, informe o horário do evento.")

    # Público do evento - Número de pessoas
    while True:
        try:
            num_pessoas = int(input("Público do evento: Número de pessoas? "))
            if num_pessoas > 0:
                briefing['num_pessoas'] = num_pessoas
                break
            else:
                print("Por favor, informe um número de pessoas válido.")
        except ValueError:
            print("Por favor, insira um número inteiro para a quantidade de pessoas.")

    # Público do evento - Perfil/público-alvo
    briefing['perfil_publico'] = input("Público do evento: Perfil/público-alvo (%mulheres x homens, faixa de idade, alta gestão, toda companhia, pessoal do escritório etc.)? ").strip()

    # Orçamento previsto
    while True:
        try:
            orcamento = float(input("Orçamento previsto em Reais (R$)? "))
            if orcamento >= 0:
                briefing['orcamento'] = orcamento
                break
            else:
                print("Por favor, informe um orçamento válido em reais (R$).")
        except ValueError:
            print("Por favor, insira um valor numérico para o orçamento.")

    # O que precisa ter
    briefing['precisa_ter'] = input("O que precisa ter? ").strip()

    # Restrições
    briefing['restricoes'] = input("Restrições: ").strip()

    # Informações complementares
    info_complementares = """
    · A PaperCare é uma empresa fabricante de papel (consumo, profissional e industrial) e produtos de cuidados pessoais (absorventes e fraldas).
    · Tem cerca de 1200 funcionários distribuídos em 2 plantas fabris (com 870 e 215 funcionários), 1 escritório (85 funcionários) e time comercial (30 funcionários). A empresa tem 57 anos de existência e tem como lema “Nosso papel é construir o futuro”.
    · Perfil do publico: 72% masculino; 28% feminino; 5% LGBTQIAP+; 80% do time das fábricas é operacional.
    · As áreas fabris funcionam 24 horas por dia. Os operadores estão divididos em 4 equipes que rodam turnos de 8 horas. A cada 5 dias essas equipes fazem um rodízio de turno, sendo que uma das equipes fica de folga.
    · O escritório, áreas administrativas e time comercial trabalham em horário comercial. Os times comerciais atuam fora do escritório.
    · Considere que o time de Comunicação e Endomarleting é reduzido para produzir e executar o evento. Composição: 3 integrantes no escritório central + 1 apoio em cada fabrica.
    """

    # Formatar a saída como um texto, incluindo as informações complementares
    texto_briefing = f"""
    Briefing do Evento:

    Tipo de Evento: {briefing['tipo_evento']}
    Tema: {briefing['tema']}
    Data/Período: {briefing['data_periodo']}
    Horário: {briefing['horario']}
    Público - Número de Pessoas: {briefing['num_pessoas']}
    Público - Perfil: {briefing['perfil_publico']}
    Orçamento Previsto(R$): {briefing['orcamento']}
    Precisa Ter: {briefing['precisa_ter']}
    Restrições: {briefing['restricoes']}

    Informações complementares:{info_complementares}
    """

    return texto_briefing

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("ASSISTENTE BUSINESS PARTY")
print("Olá sou seu assistente personalizado da PaperCare e estou aqui para auxiliá-lo a produzir os eventos da companhia.")

# Inserir lógica do sistema de agentes ################################################

# --- Obter o Briefing do Usuário ---
print("Vamos a primeira etapa. O Briefing:")
briefing = coletar_briefing()

# --- Resumo do Briefing ---
print("\n--- Resumo do Briefing ---\n")
display(to_markdown(briefing))
print("----------------------------------------------------------------")

# --- Etapa de Brainstorming ---
lista_de_ideias = agente_brainstormer(briefing)
print("\n--- Resultado da Etapa de Brainstorming ---\n")
display(to_markdown(lista_de_ideias))
print("----------------------------------------------------------------")

# --- Etapa de Planejamento ---
plano_do_evento = agente_planejador(briefing, lista_de_ideias, data_de_hoje)
print("\n--- Resultado da Etapa de Planejamento ---\n")
display(to_markdown(plano_do_evento))
print("----------------------------------------------------------------")

# --- Etapa de Criação da Comunicação ---
draft_comunicacao = agente_comunicador(briefing, plano_do_evento)
print("\n--- Resultado da Etapa de Criação da Comunicação ---\n")
display(to_markdown(draft_comunicacao))
print("----------------------------------------------------------------")

# --- Etapa de Revisão da Comunicação ---
comunicacao_revisada = agente_revisor(briefing, plano_do_evento,draft_comunicacao)
print("\n--- Resultado da Etapa de Revisão da Comunicação ---\n")
display(to_markdown(comunicacao_revisada))
print("----------------------------------------------------------------")
print(f"ASSISTENTE BUSINESS PARTY - IROTSUKA - {data_de_hoje}")
print(f"####################################################")
print(f"#  LINKEDIN:https://www.linkedin.com/in/irotsuka/  #")
print(f"#  GITHUB: https://github.com/i-otsuka             #")
print(f"####################################################")

ASSISTENTE BUSINESS PARTY
Olá sou seu assistente personalizado da PaperCare e estou aqui para auxiliá-lo a produzir os eventos da companhia.
Vamos a primeira etapa. O Briefing:
Qual o tipo de evento (Festa de final de ano, convenção, aniversário da empresa, campanha etc.)? Treinamento
Qual é o tema? IA para Produtividade no dia a dia.
Data/período do evento? De 1 a 5 de julho de 2025
Horário? das 14h as 17h
Público do evento: Número de pessoas? 50
Público do evento: Perfil/público-alvo (%mulheres x homens, faixa de idade, alta gestão, toda companhia, pessoal do escritório etc.)? Publico administrativo com perfil e nivel de conhecimento sobre IA variados.
Orçamento previsto em Reais (R$)? 5000
O que precisa ter? Apostiva e kit para os alunos e 2 instrutores.
Restrições: Local com acessibilidade para cadeirante.

--- Resumo do Briefing ---



> 
>     Briefing do Evento:
> 
>     Tipo de Evento: Treinamento
>     Tema: IA para Produtividade no dia a dia.
>     Data/Período: De 1 a 5 de julho de 2025
>     Horário: das 14h as 17h
>     Público - Número de Pessoas: 50
>     Público - Perfil: Publico administrativo com perfil e nivel de conhecimento sobre IA variados.
>     Orçamento Previsto(R$): 5000.0
>     Precisa Ter: Apostiva e kit para os alunos e 2 instrutores.
>     Restrições: Local com acessibilidade para cadeirante.
> 
>     Informações complementares:
>     · A PaperCare é uma empresa fabricante de papel (consumo, profissional e industrial) e produtos de cuidados pessoais (absorventes e fraldas). 
>     · Tem cerca de 1200 funcionários distribuídos em 2 plantas fabris (com 870 e 215 funcionários), 1 escritório (85 funcionários) e time comercial (30 funcionários). A empresa tem 57 anos de existência e tem como lema “Nosso papel é construir o futuro”.
>     · Perfil do publico: 72% masculino; 28% feminino; 5% LGBTQIAP+; 80% do time das fábricas é operacional.
>     · As áreas fabris funcionam 24 horas por dia. Os operadores estão divididos em 4 equipes que rodam turnos de 8 horas. A cada 5 dias essas equipes fazem um rodízio de turno, sendo que uma das equipes fica de folga.
>     · O escritório, áreas administrativas e time comercial trabalham em horário comercial. Os times comerciais atuam fora do escritório.
>     · Considere que o time de Comunicação e Endomarleting é reduzido para produzir e executar o evento. Composição: 3 integrantes no escritório central + 1 apoio em cada fabrica.
>     
>     

----------------------------------------------------------------

--- Resultado da Etapa de Brainstorming ---



> Okay, vamos gerar algumas ideias criativas para o treinamento "IA para Produtividade no dia a dia" da PaperCare, considerando o orçamento, público e restrições.
> 
> 
> Com base no briefing e nas informações coletadas, aqui estão 5 ideias de festa com foco em treinamento sobre IA e produtividade, adequadas ao orçamento e às restrições:
> 
> **1. Slogan:** "PaperCare + IA: Produtividade que Constroi o Futuro"
> 
> *   **Ações e Atrações:**
>     *   **Local:** Buscar um espaço de coworking ou sala de treinamento acessível na região das fábricas ou escritório central em São Paulo que se enquadre no orçamento. Priorizar locais com boa iluminação natural e que permitam flexibilidade no layout. Espaços como o Espaço Manacá ou alguns espaços de coworking podem ser boas opções dependendo dos valores e disponibilidade (considerando que estamos em 2025).
>     *   **Instrutores:** Contratar 2 especialistas em IA e produtividade com experiência em treinamento corporativo. Buscar profissionais que consigam adaptar a linguagem para diferentes níveis de conhecimento sobre IA.
>     *   **Conteúdo:** Dividir o treinamento em módulos práticos com exemplos de aplicação da IA no dia a dia da PaperCare (ex: otimização de processos, análise de dados, chatbots para suporte interno). Incluir estudos de caso e exercícios em grupo para fixar o aprendizado.
>     *   **Dinâmicas:** Promover dinâmicas interativas utilizando ferramentas de IA (ex: brainstorming com IA para geração de ideias, quizzes sobre IA com recompensas).
> *   **Materiais e Brindes:**
>     *   **Apostila:** Criar uma apostila completa com o conteúdo do treinamento, exemplos práticos e exercícios. Incluir um glossário de termos de IA para facilitar o entendimento.
>     *   **Kit:** Montar um kit para os participantes com caneta personalizada, bloco de notas ecológico, squeeze reutilizável e um pequeno guia de ferramentas de IA para produtividade.
>     *   **Brindes:** Oferecer brindes criativos relacionados ao tema, como um chaveiro personalizado com o logo da PaperCare e um QR code para um artigo sobre IA ou um fone de ouvido simples para uso nas atividades práticas.
> *   **Comunicação:**
>     *   **Convites:** Criar convites digitais personalizados com o tema do treinamento e informações relevantes.
>     *   **Banners:** Produzir banners informativos para o local do treinamento com o tema "IA para Produtividade" e o slogan do evento.
>     *   **Espaço Instagramável:** Criar um pequeno espaço instagramável com o logo da PaperCare e elementos relacionados à IA para que os participantes possam tirar fotos e compartilhar nas redes sociais.
> 
> **2. Slogan:** "Turbine Sua Produtividade: IA na Prática PaperCare"
> 
> *   **Ações e Atrações:**
>     *   **Formato:** Adotar um formato de "hackathon" com foco em soluções de IA para desafios reais da PaperCare. Dividir os participantes em grupos multidisciplinares e desafiá-los a criar protótipos de soluções inovadoras.
>     *   **Mentores:** Convidar especialistas em IA para atuarem como mentores dos grupos, oferecendo suporte técnico e orientação estratégica.
>     *   **Premiação:** Premiar os grupos vencedores com brindes especiais (ex: vale-presente, cursos online de IA).
> *   **Materiais e Brindes:**
>     *   **Kit:** Incluir no kit um "manual do hacker" com dicas de ferramentas de IA e produtividade, além de um power bank portátil para garantir que os participantes não fiquem sem bateria durante o hackathon.
>     *   **Brindes:** Oferecer adesivos personalizados com o tema do evento para que os participantes possam customizar seus laptops e cadernos.
> 
> **3. Slogan:** "Desvendando a IA: Seu Guia PaperCare para um Futuro Produtivo"
> 
> *   **Ações e Atrações:**
>     *   **Formato:** Organizar um "workshop experience" com estações interativas onde os participantes possam experimentar diferentes ferramentas de IA para produtividade (ex: geradores de texto, softwares de automação, ferramentas de análise de dados).
>     *   **Consultores:** Contratar consultores especializados em cada ferramenta para demonstrarem o uso e tirarem dúvidas dos participantes.
> *   **Materiais e Brindes:**
>     *   **Apostila:** Criar uma apostila com o passo a passo de como usar cada ferramenta de IA apresentada no workshop.
>     *   **Brindes:** Oferecer um período de teste gratuito de algumas das ferramentas de IA apresentadas para que os participantes possam continuar praticando após o treinamento.
> 
> **4. Slogan:** "IA no Seu DNA: PaperCare Rumo à Produtividade Inteligente"
> 
> *   **Ações e Atrações:**
>     *   **Formato:** Promover um treinamento gamificado com desafios e recompensas relacionados ao tema da IA e produtividade. Utilizar uma plataforma online para acompanhar o progresso dos participantes e criar um ranking.
>     *   **Leaderboard:** Exibir um leaderboard com os melhores desempenhos para incentivar a competição saudável entre os participantes.
> *   **Materiais e Brindes:**
>     *   **Kit:** Incluir no kit um "passaporte para a produtividade" com desafios e recompensas a serem conquistadas ao longo do treinamento.
>     *   **Brindes:** Oferecer medalhas personalizadas para os participantes que completarem todos os desafios do treinamento.
> 
> **5. Slogan:** "Construindo o Futuro: IA e Produtividade na PaperCare"
> 
> *   **Ações e Atrações:**
>     *   **Local:** Se possível, realizar o treinamento em um local com espaços ao ar livre para promover momentos de descontração e networking entre os participantes.
>     *   **Atividades:** Incluir atividades de team building ao ar livre com foco em resolução de problemas e criatividade.
> *   **Materiais e Brindes:**
>     *   **Kit:** Oferecer um kit com itens úteis para o dia a dia no trabalho, como um organizador de mesa, um porta-copos personalizado e um bloco de notas adesivas.
>     *   **Brindes:** Plantar uma árvore com o nome do treinamento e o logo da PaperCare como um símbolo do futuro que estão construindo juntos.
> 
> Lembre-se de adaptar as ideias ao perfil do público, ao orçamento disponível e às restrições do local. O mais importante é criar um ambiente de aprendizado divertido, interativo e que gere resultados práticos para a PaperCare.
> 


----------------------------------------------------------------

--- Resultado da Etapa de Planejamento ---



...

----------------------------------------------------------------

--- Resultado da Etapa de Criação da Comunicação ---



> Perfeito! Com base no briefing detalhado e no plano do evento, elaborei os textos para as peças de comunicação, focando em leitura rápida, engajamento e alinhamento com a PaperCare:
> 
> **Título e Slogan Padrão para Todas as Peças:**
> 
> *   **Título:** IA para Produtividade na PaperCare
> *   **Slogan:** "PaperCare + IA: Produtividade que Constroi o Futuro"
> 
> **Textos para as Peças de Comunicação:**
> 
> 1.  **Convite Digital Personalizado (E-mail Marketing/WhatsApp):**
> 
> **Assunto:** Prepare-se para o futuro: Treinamento de IA na PaperCare!
> 
> **Corpo do E-mail:**
> 
> Olá, [Nome do Funcionário]!
> 
> A PaperCare está investindo no futuro e convidando você para o treinamento exclusivo: "IA para Produtividade na PaperCare"!
> 
> De 01 a 05 de julho, das 14h às 17h, aprenda a usar a Inteligência Artificial para otimizar seu trabalho, aumentar a produtividade e construir um futuro mais eficiente para nossa empresa.
> 
> **O que você vai aprender:**
> 
> *   Conceitos básicos de IA
> *   Ferramentas práticas para o dia a dia
> *   Como a IA pode transformar a PaperCare
> *   Ética e responsabilidade no uso da IA
> 
> **Inscreva-se agora:** [Link para Inscrição]
> 
> Vagas Limitadas!
> 
> Nosso papel é construir o futuro, e a IA é uma ferramenta poderosa para isso.
> 
> Atenciosamente,
> 
> Time PaperCare
> 
> **Hashtags:** #PaperCare #IAparaProdutividade #Inovação #FuturoDoTrabalho #Treinamento
> 
> 2.  **Cartazes e Banners (Fábricas e Escritório):**
> 
> **Título:** IA para Produtividade na PaperCare
> 
> **Slogan:** "PaperCare + IA: Produtividade que Constroi o Futuro"
> 
> **Texto:**
> 
> Atenção, time PaperCare!
> 
> Participe do nosso treinamento de IA e descubra como a Inteligência Artificial pode transformar o seu dia a dia!
> 
> *   **Data:** 01 a 05 de Julho
> *   **Horário:** 14h às 17h
> *   **Local:** [Local do Evento]
> 
> **Inscreva-se já!** [QR Code ou Link Curto]
> 
> Seja parte da revolução da produtividade na PaperCare!
> 
> **Hashtags:** #PaperCare #IAparaProdutividade #Inovação #Eficiência #Treinamento
> 
> 3.  **Fundo de Tela (Computadores):**
> 
> *   Design visualmente atraente com o logo da PaperCare e imagens relacionadas à IA.
> *   **Título:** IA para Produtividade na PaperCare
> *   **Slogan:** "PaperCare + IA: Produtividade que Constroi o Futuro"
> *   **Texto:** Prepare-se para o treinamento de IA que vai transformar a PaperCare! [Data e Link]
> 
> 4.  **Página na Intranet:**
> 
> **Título:** IA para Produtividade na PaperCare
> 
> **Slogan:** "PaperCare + IA: Produtividade que Constroi o Futuro"
> 
> **Conteúdo:**
> 
> *   **Descrição:** Descubra como a Inteligência Artificial pode impulsionar a sua produtividade e otimizar os processos da PaperCare.
> *   **Programação Completa:** Detalhes de cada dia do treinamento, com os temas e horários.
> *   **Palestrantes:** Informações sobre os instrutores, suas qualificações e experiência.
> *   **Inscrições:** Link para o formulário de inscrição.
> *   **Notícias e Atualizações:** Mantenha-se informado sobre o evento.
> *   **FAQ:** Respostas para as perguntas mais frequentes.
> 
> 5.  **Vídeos e Slides para as TVs Corporativas:**
> 
> *   **Vídeos Curtos (15-30 segundos):**
>     *   Depoimentos de funcionários sobre suas expectativas em relação ao treinamento.
>     *   Animações mostrando como a IA pode ser aplicada em diferentes áreas da PaperCare.
> *   **Slides:**
>     *   Título e Slogan em destaque.
>     *   Datas, horários e local do treinamento.
>     *   Benefícios de participar do treinamento.
>     *   Chamada para inscrição com QR Code ou link curto.
> 
> 6.  **Posts para Redes Sociais (Se Aplicável):**
> 
> *   Adapte os textos e as imagens para cada plataforma (LinkedIn, Instagram, etc.).
> *   Use vídeos curtos e chamativos para atrair a atenção.
> *   Incentive o compartilhamento e o engajamento com perguntas e enquetes.
> 
> **Exemplo de Post para LinkedIn:**
> 
> A PaperCare está investindo no futuro da produtividade! 🚀
> 
> Estamos oferecendo um treinamento exclusivo sobre "IA para Produtividade na PaperCare". Descubra como a Inteligência Artificial pode transformar o seu trabalho e impulsionar a nossa empresa!
> 
> [Link para Inscrição]
> 
> #PaperCare #IAparaProdutividade #Inovação #Liderança #TransformaçãoDigital
> 
> **Considerações Adicionais:**
> 
> *   Mantenha a identidade visual da PaperCare em todas as peças de comunicação.
> *   Use uma linguagem clara, simples e direta.
> *   Destaque os benefícios do treinamento para os funcionários.
> *   Crie um senso de urgência para incentivar as inscrições.
> *   Monitore os resultados das campanhas de comunicação para otimizar as estratégias.
> 
> Com esses textos e peças de comunicação, a PaperCare estará pronta para divulgar o treinamento de IA de forma eficaz e engajadora, preparando seus funcionários para o futuro!
> 


----------------------------------------------------------------

--- Resultado da Etapa de Revisão da Comunicação ---



> Aprovado! Podem seguir.


----------------------------------------------------------------
ASSISTENTE BUSINESS PARTY - IROTSUKA - 17/05/2025
####################################################
#  LINKEDIN:https://www.linkedin.com/in/irotsuka/  #
#  GITHUB: https://github.com/i-otsuka             #
####################################################
